# GlobusWorld 2018 JupyterHub Tutorial

This tutorial will show you how to pull Globus Auth tokens from the Jupyter Notebook Server environment and use them to interact with different REST APIs secured with Globus Auth. The code in this notebook are very pedantic for clarity at each step; much of this code could be encapsulated in Python packages to simplify the work in the notebook. Research project could also add more semantically meaningful layers for their users (e.g., `MyData.download()` rather 
than `request.get('https://example.org/mydata.csv`)`.

## Import Packages

In [1]:
# These are to get the tokens
import os
import pickle
import base64

# Much of what we're dealing with is JSON
import json

# We're going to be making explicit HTTPS calls
import requests

# This is to work with data for our example
import numpy
import matplotlib
from io import StringIO

## Get Tokens

The tokens are passed into the environment `base64` encoded pickled Python dictionary assigned to the `GLOBUS_DATA` variable. We'll grab the variable and unpack it. 

In [2]:
globus_env_data = os.getenv('GLOBUS_DATA')

In [3]:
# This is the raw base64 encoded data
print(globus_env_data)

gAN9cQAoWAkAAABjbGllbnRfaWRxAVgkAAAAZjNhNjg3YTMtZWJkYy00ZWQ2LWFmNWItYzE2YTJkMmEzZmQ5cQJYBgAAAHRva2Vuc3EDfXEEKFgPAAAAYXV0aC5nbG9idXMub3JncQV9cQYoWAUAAABzY29wZXEHWBQAAABwcm9maWxlIG9wZW5pZCBlbWFpbHEIWAwAAABhY2Nlc3NfdG9rZW5xCVhkAAAAQWc0Mnd6RTZkajdZdjYyV28xZ1gwWU5ERG5Fbks3bTRxeDVNazlWQmE4SjRuZTRXb1d0N0MxOHZ3VkphMXcxTnZwbDJ6enBhVmd6ejZ5Q2p2SzVkR1VWS25FY3lnQjlUNFlWeXEKWA0AAAByZWZyZXNoX3Rva2VucQtOWAoAAAB0b2tlbl90eXBlcQxYBgAAAEJlYXJlcnENWBIAAABleHBpcmVzX2F0X3NlY29uZHNxDkq4MuFadVgTAAAAcGV0cmVsX2h0dHBzX3NlcnZlcnEPfXEQKGgHWEcAAABodHRwczovL2F1dGguZ2xvYnVzLm9yZy9zY29wZXMvNTZjZWFjMjktZTk4YS00NDBhLWE1OTQtYjQxZTdhMDg0YjYyL2FsbHERaAlYWgAAAEFnT1Y3b3Yyb0dtbmRCUXlYa2xsSk0zMjVLT1AzN3d5eTJLWDdPUDVwNHJKcGRQYWpLaW5DQk5NbE9qak5NN283eXFXN3JtcDZORDJ6bUZlOG12YTRUNTEwMXESaAtOaAxYBgAAAEJlYXJlcnETaA5KuDLhWnVYFwAAAHRyYW5zZmVyLmFwaS5nbG9idXMub3JncRR9cRUoaAdYMQAAAHVybjpnbG9idXM6YXV0aDpzY29wZTp0cmFuc2Zlci5hcGkuZ2xvYnVzLm9yZzphbGxxFmgJWFoAAABBZzBOUHl6Qmp4TTNrZWFwYmJ3cDZsWFdQcjU0WDY1SjNiZTU3dmpOS01wcnFXVzY0WVQ3QzNnZDM5SzBn

In [4]:
# Now we have the pickled tokens
pickled_tokens = base64.b64decode(globus_env_data)

In [5]:
# Unpickle and get the dictionary
tokens = pickle.loads(pickled_tokens)
isinstance(tokens, dict)

True

## Look Inside the Tokens

Depending on the JupyterHub configuration, there will be different numbers of tokens. For this tutorial we have our identity token as __[JSON Web Token (JWT)](https://tools.ietf.org/html/rfc7519)__, and then access tokens for different Resource Servers and scopes, like getting our profile from Globus Auth.

In [6]:
print(json.dumps(tokens, indent=4, sort_keys=True))

{
    "client_id": "f3a687a3-ebdc-4ed6-af5b-c16a2d2a3fd9",
    "id_token": {
        "at_hash": "GFdDCsqqFNG26GAKOwlppw7mk3L3gAFxt1LO79zecQE",
        "aud": "f3a687a3-ebdc-4ed6-af5b-c16a2d2a3fd9",
        "email": "rick@globus.org",
        "exp": 1524708024,
        "iat": 1524535224,
        "identity_provider": "41143743-f3c8-4d60-bbdb-eeecaba85bd9",
        "identity_provider_display_name": "Globus ID",
        "iss": "https://auth.globus.org",
        "name": "Rick Wagner",
        "organization": "Globus",
        "preferred_username": "rpwagner@globusid.org",
        "sub": "c4e16e62-d274-11e5-b9e9-cf0225068ce8"
    },
    "tokens": {
        "auth.globus.org": {
            "access_token": "Ag42wzE6dj7Yv62Wo1gX0YNDDnEnK7m4qx5Mk9VBa8J4ne4WoWt7C18vwVJa1w1Nvpl2zzpaVgzz6yCjvK5dGUVKnEcygB9T4YVy",
            "expires_at_seconds": 1524708024,
            "refresh_token": null,
            "scope": "profile openid email",
            "token_type": "Bearer"
        },
        "petrel_

## Using the Tokens

Now we can get started talking to different servers. For the purpose of this tutorial we're going to show how the tokens can be passed as HTTP headers. Much of this can also be done with the __[Globus Python SDK] (http://globus-sdk-python.readthedocs.io/en/stable/)__.

### User Info

First, let's use our `auth.globus.org` token to get our __[OAuth2 user information](https://docs.globus.org/api/auth/reference/#get_or_post_v2_oauth2_userinfo_resource)__. We assemble the header with the appropriate access token and do an HTTP `GET` on the resource. (N.B. We're using HTTPS even though we refer to the HTTP methods.) 

In [7]:
# Create the header
headers = {'Authorization':'Bearer '+ tokens['tokens']['auth.globus.org']['access_token']}

# Get the user info as JSON
user_info = requests.get('https://auth.globus.org/v2/oauth2/userinfo',headers=headers).json()

# Look at the response
print(json.dumps(user_info, indent=4, sort_keys=True))

{
    "email": "rick@globus.org",
    "identity_provider": "41143743-f3c8-4d60-bbdb-eeecaba85bd9",
    "identity_provider_display_name": "Globus ID",
    "name": "Rick Wagner",
    "organization": "Globus",
    "preferred_username": "rpwagner@globusid.org",
    "sub": "c4e16e62-d274-11e5-b9e9-cf0225068ce8"
}


### Identities

This is the __[Globus Auth API resource for identities](https://docs.globus.org/api/auth/reference/#v2_api_identities_resources)__. We use a similar process, but we're doing a `GET` on a specific identity, our own.

In [8]:
identity = requests.get('https://auth.globus.org/v2/api/identities/' + user_info['sub'],headers=headers).json()
print(json.dumps(identity, indent=4, sort_keys=True))

{
    "identity": {
        "email": "rick@globus.org",
        "id": "c4e16e62-d274-11e5-b9e9-cf0225068ce8",
        "identity_provider": "41143743-f3c8-4d60-bbdb-eeecaba85bd9",
        "name": "Rick Wagner",
        "organization": "Globus",
        "status": "used",
        "username": "rpwagner@globusid.org"
    }
}


## A Minimal Workflow

Let's replicate a basic workflow:

1. Pull down some data
1. Plot it
1. Save the plot to a webserver
1. Share the link

### Pulling Down a CSV File

For our data, we'll replicate some of the flow from the Globus __[Modern Research Data Portal](https://mrdp.globus.org)__. In particular, we'll get the climate data for Las Vegas from 1952. This is a CSV file with column names in the first row.

In [9]:
# GET the CSV from the publicly acessbible HTTPS GCS endpoint
vegas_climate_csv = requests.get('https://tutorial-https-endpoint.globus.org/portal/catalog/dataset_las/1952.csv').text

In [10]:
# Convert the CSV into a Numpy array
vegas_climate_array = numpy.genfromtxt(StringIO(vegas_climate_csv), delimiter=",", names=True)
print(vegas_climate_array.shape)
print()
print(vegas_climate_array[0])

(366,)

(nan, nan, 659., 36.08333, -115.16667, 19520101., 0., 44., -50.)


<Response [200]>


### Plot the Data

In [13]:
import matplotlib.pyplot as plt
import numpy as np

t = np.arange(0.0, 2.0, 0.01)
s = 1 + np.sin(2*np.pi*t)
plt.plot(t, s)

plt.xlabel('time (s)')
plt.ylabel('voltage (mV)')
plt.title('About as simple as it gets, folks')
plt.grid(True)
plt.savefig("test.png")
plt.show()

<Figure size 640x480 with 1 Axes>

### Save the Plot

For these Globus Transfer API interactions we'll be using the transfer token. Also, the Globus Transfer API does not support __[`application/x-www-form-urlencoded`](https://docs.globus.org/api/transfer/overview/#document_formats)__ data so we'll be explicit about the JSON we pass.

At this point, we'll:

1. Activate the endpoint we're using for the tutorial
1. Create a directory for our file
1. `PUT` our plot there
1. Generate a link to our plot and view it

Note, the calls to Transfer can be made via the SDK, with argument validation, etc. For this tutorial we're sticking with direct HTTP request for pedagogical purposes.

In [37]:
base_url = 'https://transfer.api.globusonline.org/v0.10'
endpoint_uuid = 'e56c36e4-1063-11e6-a747-22000bf2d559'

# Create the header
headers = {'Authorization':'Bearer '+ tokens['tokens']['transfer.api.globus.org']['access_token'],
          "Content-Type" : "application/json"}

In [31]:
# Grab our username (which includes a hash to avoid collision)
username = os.getenv('JUPYTERHUB_USER')
print("My user name is " + username)

My user name is rpwagnermm2gkmjwmu3dellegi3til


In [38]:
# Autoactivate the endpoint
resp = requests.post(base_url + '/endpoint/' + endpoint_uuid + '/autoactivate',
                    headers=headers)
print(resp.status_code)
print(resp.text)

200
{
  "DATA": [], 
  "DATA_TYPE": "activation_result", 
  "code": "AutoActivated.GlobusOnlineCredential", 
  "endpoint": "petrel#testbed", 
  "expire_time": "2018-04-25 02:54:49+00:00", 
  "expires_in": -1, 
  "length": 0, 
  "message": "Endpoint activated successfully using Globus Online credentials.", 
  "oauth_server": null, 
  "request_id": "SeewkpYHO", 
  "resource": "/endpoint/e56c36e4-1063-11e6-a747-22000bf2d559/autoactivate", 
  "subject": "/C=US/O=Globus Consortium/OU=Globus Connect User/CN=rpwagner"
}


In [41]:
# Call the Transfer API to make the directory
mkdir_json = json.dumps({ "DATA_TYPE": "mkdir",
              "path": "/test/jhtutorial/users/" + username })

resp = requests.post(base_url + '/endpoint/' + endpoint_uuid + '/mkdir',
                    headers=headers, data=mkdir_json)
print(resp.status_code)
print(resp.text)

502
{
  "code": "ExternalError.MkdirFailed.Exists", 
  "message": "Path already exists, Error (mkdir)\nEndpoint: petrel#testbed (e56c36e4-1063-11e6-a747-22000bf2d559)\nServer: petreldtn5.pub.jlse.anl.gov:2811\nMessage: Path '/test/jhtutorial/users/rpwagnermm2gkmjwmu3dellegi3til' already exists\n", 
  "request_id": "7ay2v3BFH", 
  "resource": "/endpoint/e56c36e4-1063-11e6-a747-22000bf2d559/mkdir"
}


### Put the Plot
This GCS endpoint supports HTTPS and we can `PUT` the plot image file there directly. (If it was larger or many files we might want to use a Tranfer request.)

In [43]:
# Create the header
headers = {'Authorization':'Bearer '+ tokens['tokens']['petrel_https_server']['access_token']}

# Pass the file pointer reference to the requests library for the PUT
image_data = open('test.png', 'rb').read()

# Get the user info as JSON
resp = requests.put('https://testbed.petrel.host/test/jhtutorial/users/' + username + '/test2.png',
                    headers=headers, data=image_data, allow_redirects=False)
print(resp.status_code)

200



### The Link

Let's look at a link to the file and call it a day.

In [44]:
print('https://testbed.petrel.host/test/jhtutorial/users/' + username + '/test2.png')

https://testbed.petrel.host/test/jhtutorial/users/rpwagnermm2gkmjwmu3dellegi3til/test2.png
